In [1]:
import numpy as np
import pandas as pd
import random
import keras
import tensorflow as tfw
import os
from matplotlib import pyplot as plt
from PIL import Image

In [1]:
samples=20000
df=pd.read_csv("train.csv")
df

In [ ]:
df1=pd.read_csv("train (1).csv")
df1

In [ ]:
df1.drop(labels="url")

In [ ]:
df=df.loc[df["id"].str.startswith(('b1','00'),na=False),:]
num_classes=len(df["landmark_id"].unique())
num_data=len(df)

In [ ]:
num_classes

In [ ]:
num_data

In [ ]:
data=pd.DataFrame(df["landmark_id"].value_counts())
data.reset_index(inplace=True)

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.columns=['landmark_id','count']

In [ ]:
data['count'].describe()

In [ ]:
data

In [ ]:
data['count'].describe()

In [ ]:
plt.hist(data['count'],100,range=(0,64),label='tset')

In [ ]:
data['count'].between(0,5).sum()

In [ ]:
data['count'].between(5,10).sum()

In [ ]:
data['count'].between(10,15).sum()

In [ ]:
plt.hist(df['landmark_id'], bins=len(df['landmark_id'].unique()))

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labelencoder.fit(df['landmark_id'])

In [ ]:
df

In [ ]:
def encode_label(label):
    return labelencoder.transform(label)

In [ ]:
def decode_label(label):
    return labelencoder.inverse_transform(label)

In [ ]:
def get_img_from_num(num,df):
    fname,label=df.iloc[num,:]
    f1=fname[0]
    f2=fname[1]
    f3=fname[2]
    path=os.path.join("./images",f1,f2,f3,fname)
    img=cv2.imread(path)
    return img,label

In [ ]:
fig=plt.figure(figsize=(16,16))
for i in range(1,7):
    rimg=random.choices(os.listdir("./images"),k=3)
    print(rimg)
    folder="./images" + "/" +"0"+"/"+"i"+"/"+rimg[2]
    random_img=random.choice(os.listdir(folder))
    img=np.array(Image.open(folder+"/"+random_img))
    fig.add_subplot(1,4,i)
    plt.imshow(img)
    plt.axis('off')
plt.show
    

# Build the model

In [ ]:
import tensorflow as tfw
from keras.applications.vgg19 import VGG19
from keras.layers import *
from keras import Sequential
tfw.compat.v1.disable_eager_execution

In [ ]:
learning_rate = 0.0001
decay_speed = 1e-6
momentum = 0.09
loss_function = "sparse_categorical_crossentropy"
source_model = VGG19(weights=None)
drop_layer = Dropout(0.5)
drop_layer2 = Dropout(0.5)

In [ ]:
model=Sequential()
for layer in source_model.layers[:-1]:
    if layer == source_model.layers[-25]:
        model.add(BatchNormalization())
    model.add(layer)
model.add(Dense(num_classes,activation="softmax"))
model.summary()

In [ ]:
from keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(learning_rate=learning_rate),
              loss=loss_function,
              metrics=['accuracy'])


In [ ]:
def image_reshape(img,size):
    return cv2.resize(img,size)

In [ ]:
def get_batch(dataframe,start,batch_size):
    image_array=[]
    label_array=[]
    end_image=start+batch_size
    if(end_image)>len(dataframe):
        end_image=len(dataframe)
    for idx in range(start,end_image):
        n=idx
        img,label=get_img_from_num(n,dataframe)
        img=image_reshape(img,(224,224))/255.0
        image_array.append(img)
        label_array.append(label)
    label_array=encode_label(label_array)
    return np.array(image_array),np.array(label_array)

In [ ]:
batch_size=64
epoch_shuffle=True
weight_classes=True
epochs=1

train,val = np.split(df.sample(frac=1),[int(0.8*len(df))])
print(len(train))
print(len(val))

In [ ]:
for e in range(epochs):
    print("Epoch :" + str(e+1) + "/" + str(epochs))
    if epoch_shuffle:
        train=train.sample(frac=1)
    for it in range(int(np.ceil(len(train)/batch_size))):
        X_train,y_train=get_batch(train,it*batch_size,batch_size)
        
        model.fit(X_train, y_train, batch_size=batch_size)

In [ ]:
batch_size = 16
errors = 0
good_preds = []
bad_preds = []

for it in range(int(np.ceil(len(val) / batch_size))):
    x_val, y_val = get_batch(val, it * batch_size, batch_size)
    result = model.predict(x_val)
    cla = np.argmax(result, axis=1)
    for idx, res in enumerate(result):
        if cla[idx] != y_val[idx]:
            errors += 1
            bad_preds.append([batch_size * it + idx, cla[idx], res[cla[idx]]])
        else:
            good_preds.append([batch_size * it + idx, cla[idx], res[cla[idx]]])
